In [24]:

import random 
import math

random.seed(123456)

R = 6371000.0
def distance(p1,p2):
    return math.sqrt((p1[0] - p2[0])**2 + (math.cos(p1[0])*(p1[1] - p2[1]))**2) * R
    
def get_mean(points):
    np = len(points)
    sump = [0,0]
    if np == 0:
        return sump
    for p in points:
        sump[0] += p[0]
        sump[1] += p[1]
    sump[0] = sump[0]/np
    sump[1] = sump[1]/np
    return sump

def associate(centers, points, associated):
    nchanged = 0
    for ip, p in enumerate(points):
        closest = None
        mindist = 1e10
        for ic, c in enumerate(centers):
            dist = distance(p, c)
            if dist < mindist:
                mindist = dist
                closest = ic
        if closest is None: print("Something wrong. Look into associate function!")
        if associated[ip] != closest:
            associated[ip] = closest
            nchanged += 1
    return float(nchanged)/len(points)

def get_centroids(points, associated, kcls):
    center_points = [[] for _ in range(kcls)]
    for ip, p in enumerate(points):
        center_points[associated[ip]].append(p)
    centroids = []
    for icls in range(kcls):
        mean = get_mean(center_points[icls])
        centroids.append(mean)
    return centroids

def kmeans(points, kcls, max_iter=1000, min_shift_frac=0.01):
    
    npoints = len(points)
    if npoints <= kcls**2: kcls = int(npoints**0.5)
    
    centers = random.sample(points, kcls)
    associated = [0 for _ in points]
    for iter in range(max_iter):
        changed_fraction = associate(centers, points, associated)
        if changed_fraction <= min_shift_frac:
            print("changed_fraction({}) reached to min_shift_frac({})".format(changed_fraction, min_shift_frac))
            break
        if iter == max_iter-1:
            print("max_iter({}) reached.".format(iter))
    return associated


In [25]:
data = []
for iline, line in enumerate(open('crash2.csv', 'r')):
    if iline == 0: continue
    p = line.strip().split(",")
    data.append((float(p[0]),float(p[1])))

kcls = 4
associated = kmeans(data, kcls, max_iter=500, min_shift_frac=0.01)
centroids = get_centroids(data, associated, kcls)

changed_fraction(0.0) reached to min_shift_frac(0.01)


In [26]:
centroids

[[41.893113077099976, -87.6512527903],
 [41.89503547777777, -87.65385070714285],
 [41.89152598206895, -87.64898402718391],
 [41.893799806636636, -87.65704604555538]]

In [27]:
associated

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 0,
 1,
 1,
 3,
 3,
 1,
 3,
 2,
 1,
 3,
 3,
 3,
 0,
 3,
 3,
 0,
 3,
 2,
 1,
 2,
 3,
 1,
 2,
 0,
 2,
 1,
 2,
 3,
 1,
 3,
 2,
 3,
 2,
 1,
 1,
 3,
 3,
 1,
 3,
 1,
 0,
 0,
 3,
 2,
 2,
 1,
 3,
 3,
 3,
 0,
 3,
 0,
 3,
 3,
 3,
 0,
 2,
 0,
 3,
 3,
 1,
 0,
 2,
 3,
 1,
 1,
 3,
 3,
 0,
 3,
 3,
 2,
 3,
 0,
 2,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 2,
 3,
 2,
 3,
 3,
 0,
 3,
 0,
 3,
 0,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 3,
 2,
 3,
 0,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 1,
 3,
 3,
 3,
 0,
 3,
 3,
 2,
 3,
 2,
 1,
 3,
 2,
 0,
 3,
 3,
 2,
 1,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 2,
 0,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 2,
 0,
 0,
 2,
 3,
 2,
 1,
 3,
 3,
 2,
 2,
 3,
 3,
 0,
 3,
 0,
 0,
 2,
 2,
 0,
 3,
 1,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 0,
 2,
 2,
 3,
 3,
 3,
 0,
 3,
 2,
 3,
 1,
 0,
 0,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 3,
 2,
 3,
 1,
 3,
 2,
 2,
 1,
 2,
 0,
 3,
 3,
 3,
 3,
 2,
 1,
 3,
 3,
 2,


In [28]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [29]:
average_max = -1
kcls_max = None

for i in range(3,50):
    kcls = i
    associated = kmeans(data, kcls, max_iter=500, min_shift_frac=0.01)
    centroids = get_centroids(data, associated, kcls)
    silhouette_avg = silhouette_score(data,associated)
    print("For kcls =", kcls,
          "The average silhouette_score is :", silhouette_avg)
    
    print(silhouette_avg)
    if silhouette_avg > average_max:
        average_max = silhouette_avg
        kcls_max = kcls
print(kcls_max, average_max)

changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 3 The average silhouette_score is : 0.4531823797590823
0.4531823797590823
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 4 The average silhouette_score is : 0.4970403684136513
0.4970403684136513
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 5 The average silhouette_score is : 0.3392964407003656
0.3392964407003656
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 6 The average silhouette_score is : 0.4452379710857146
0.4452379710857146
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 7 The average silhouette_score is : 0.33410534266746766
0.33410534266746766
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 8 The average silhouette_score is : 0.27163210556763034
0.27163210556763034
changed_fraction(0.0) reached to min_shift_frac(0.01)
For kcls = 9 The average silhouette_score is : 0.3496212118759795
0.3496212118759795
changed_fraction(0.0) r

In [30]:
first_part = """
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="initial-scale=1.0, width=device-width" />
<link rel="stylesheet" type="text/css" href="https://js.api.here.com/v3/3.0/mapsjs-ui.css?dp-version=1542186754" />
<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-core.js"></script>
<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-service.js"></script>
<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-ui.js"></script>
<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-mapevents.js"></script>

</head>
<body>
  <div id="map" style="width: 100%; height: 800px; background: grey" />
  <script  type="text/javascript" charset="UTF-8" >
    

/**
 * Adds markers to the map highlighting the locations of the captials of
 * France, Italy, Germany, Spain and the United Kingdom.
 *
 * @param  {H.Map} map      A HERE Map instance within the application
 */
function addMarkersToMap(map) {
  //var parisMarker = new H.map.Marker({lat:48.8567, lng:2.3508});
  //map.addObject(parisMarker);
}

/**
 * Boilerplate map initialization code starts below:
 */

//Step 1: initialize communication with the platform
var platform = new H.service.Platform({
  app_id: 'devportal-demo-20180625',
  app_code: '9v2BkviRwi9Ot26kp2IysQ',
  useHTTPS: true
});
var pixelRatio = window.devicePixelRatio || 1;
var defaultLayers = platform.createDefaultLayers({
  tileSize: pixelRatio === 1 ? 256 : 512,
  ppi: pixelRatio === 1 ? undefined : 320
});

//Step 2: initialize a map - this map is centered over Europe
var map = new H.Map(document.getElementById('map'),
  defaultLayers.normal.map,{
  center: {lat:41, lng:-87},
  zoom: 4,
  pixelRatio: pixelRatio
});

//Step 3: make the map interactive
// MapEvents enables the event system
// Behavior implements default interactions for pan/zoom (also on mobile touch environments)
var behavior = new H.mapevents.Behavior(new H.mapevents.MapEvents(map));

// Create the default UI components
var ui = H.ui.UI.createDefault(map, defaultLayers);

"""

last_part = """

// Now use the map as required...
addMarkersToMap(map);
  </script>
</body>
</html>
"""

In [31]:
first_part

'\n<!DOCTYPE html>\n<html>\n<head>\n<meta name="viewport" content="initial-scale=1.0, width=device-width" />\n<link rel="stylesheet" type="text/css" href="https://js.api.here.com/v3/3.0/mapsjs-ui.css?dp-version=1542186754" />\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-core.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-service.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-ui.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-mapevents.js"></script>\n\n</head>\n<body>\n  <div id="map" style="width: 100%; height: 800px; background: grey" />\n  <script  type="text/javascript" charset="UTF-8" >\n    \n\n/**\n * Adds markers to the map highlighting the locations of the captials of\n * France, Italy, Germany, Spain and the United Kingdom.\n *\n * @param  {H.Map} map      A HERE Map instance within the application\n */\nfunction addMarkersT

In [32]:
last_part

'\n\n// Now use the map as required...\naddMarkersToMap(map);\n  </script>\n</body>\n</html>\n'

In [33]:
def construct_middle_part(points):
    """
    input
    
    points: [(lat,lng), (lat,lng), ...]
    
    output
    
    string of
        var parisMarker = new H.map.Marker({lat:48.8567, lng:2.3508});
        map.addObject(parisMarker);
  
    """

    output_str = "var IconClst0 = new H.map.Icon(\"https://image.flaticon.com/icons/svg/252/252025.svg\", {size: {w: 32, h: 32}});"
    output_str += "var IconClst1 = new H.map.Icon(\"https://image.flaticon.com/icons/svg/58/58960.svg\", {size: {w: 32, h: 32}});"
    output_str += "var IconClst2 = new H.map.Icon(\"https://image.flaticon.com/icons/svg/149/149060.svg\", {size: {w: 32, h: 32}});"
    output_str += "var IconClst3 = new H.map.Icon(\"https://image.flaticon.com/icons/svg/1502/1502944.svg\", {size: {w: 32, h: 32}});"
    
    associated = kmeans(points, 4, max_iter=500, min_shift_frac=0.01)
    
    i = 0
    
    for p in points:
        output_str += """
            var parisMarker = new H.map.Marker({lat:%f, lng:%f}, {icon: IconClst%i});
            map.addObject(parisMarker);
        """ % (p[0], p[1], associated[i])
        i += 1
    return output_str

In [34]:
import csv

points = []
for iline, line in enumerate(open('crash2.csv', 'r')):
    if iline == 0: continue
    p = line.strip().split(",")
    points.append((float(p[0]),float(p[1])))

print(points)

[(41.89467657, -87.65732504), (41.89467657, -87.65732504), (41.89467657, -87.65732504), (41.89613412, -87.6565956), (41.89238383, -87.65772697), (41.89165427, -87.65725278), (41.89294663, -87.65757787), (41.89542528, -87.6551759), (41.89129371, -87.64768059), (41.89121804, -87.65468948), (41.89599599, -87.6524789), (41.89324215, -87.65570519), (41.89539756, -87.65576524), (41.89460555, -87.65732313), (41.8929702, -87.65598773), (41.8911581, -87.65911749), (41.89124229, -87.65235587), (41.89584197, -87.65481897), (41.89283282, -87.65768284), (41.89587084, -87.65607103), (41.89546625, -87.65576637), (41.89120814, -87.65573434), (41.89460555, -87.65732313), (41.89442373, -87.65761242), (41.89313364, -87.64780859), (41.8955426, -87.65542425), (41.89133821, -87.64894251), (41.89450195, -87.65339161), (41.89122956, -87.6535221), (41.89613873, -87.65634571), (41.89428162, -87.65350006), (41.89129371, -87.64768059), (41.89395876, -87.64784119), (41.89278268, -87.64776455), (41.89591575, -87.65

In [35]:
middle = construct_middle_part(points)
middle

changed_fraction(0.0) reached to min_shift_frac(0.01)


'var IconClst0 = new H.map.Icon("https://image.flaticon.com/icons/svg/252/252025.svg", {size: {w: 32, h: 32}});var IconClst1 = new H.map.Icon("https://image.flaticon.com/icons/svg/58/58960.svg", {size: {w: 32, h: 32}});var IconClst2 = new H.map.Icon("https://image.flaticon.com/icons/svg/149/149060.svg", {size: {w: 32, h: 32}});var IconClst3 = new H.map.Icon("https://image.flaticon.com/icons/svg/1502/1502944.svg", {size: {w: 32, h: 32}});\n            var parisMarker = new H.map.Marker({lat:41.894677, lng:-87.657325}, {icon: IconClst3});\n            map.addObject(parisMarker);\n        \n            var parisMarker = new H.map.Marker({lat:41.894677, lng:-87.657325}, {icon: IconClst3});\n            map.addObject(parisMarker);\n        \n            var parisMarker = new H.map.Marker({lat:41.894677, lng:-87.657325}, {icon: IconClst3});\n            map.addObject(parisMarker);\n        \n            var parisMarker = new H.map.Marker({lat:41.896134, lng:-87.656596}, {icon: IconClst3});\n

In [36]:
whole = first_part + middle + last_part
whole

'\n<!DOCTYPE html>\n<html>\n<head>\n<meta name="viewport" content="initial-scale=1.0, width=device-width" />\n<link rel="stylesheet" type="text/css" href="https://js.api.here.com/v3/3.0/mapsjs-ui.css?dp-version=1542186754" />\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-core.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-service.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-ui.js"></script>\n<script type="text/javascript" src="https://js.api.here.com/v3/3.0/mapsjs-mapevents.js"></script>\n\n</head>\n<body>\n  <div id="map" style="width: 100%; height: 800px; background: grey" />\n  <script  type="text/javascript" charset="UTF-8" >\n    \n\n/**\n * Adds markers to the map highlighting the locations of the captials of\n * France, Italy, Germany, Spain and the United Kingdom.\n *\n * @param  {H.Map} map      A HERE Map instance within the application\n */\nfunction addMarkersT

In [37]:
from IPython.display import HTML
display(HTML(whole))

In [38]:
f = open("whole_map2.html", "w")

f.write(whole)

f.close()

In [39]:
import gmaps